# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [125]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [126]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
df.prestige=df.prestige.astype(int)
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61         3
1      1  660.0  3.67         3
2      1  800.0  4.00         1
3      1  640.0  3.19         4
4      0  520.0  2.93         4


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [127]:
# frequency table for prestige and whether or not someone was admitted
df.prestige.value_counts().sort_index(0)

1     61
2    148
3    121
4     67
Name: prestige, dtype: int64

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [128]:
dummy_ranks=pd.get_dummies(df['prestige'], drop_first=True, prefix="prestige")
print dummy_ranks.head()

   prestige_2  prestige_3  prestige_4
0           0           1           0
1           0           1           0
2           0           0           0
3           0           0           1
4           0           0           1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [130]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           1           0
1      1  660.0  3.67           0           1           0
2      1  800.0  4.00           0           0           0
3      1  640.0  3.19           0           0           1
4      0  520.0  2.93           0           0           1


In [131]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
print pd.crosstab(df["prestige"], df["admit"])
print pd.crosstab(df["prestige"], df["admit"]).ix[1]

admit      0   1
prestige        
1         28  33
2         95  53
3         93  28
4         55  12
admit
0    28
1    33
Name: 1, dtype: int64


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [149]:
33/28=1.179

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [148]:
(53+28+12)/(95+93+55)=93/243=31/81=0.383

#### 3.3 Calculate the odds ratio

In [146]:
(33/28)/(31/81)=1.487

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of getting admitted to grad school for students from #1 ranked college is 1.487 times the odds of admission for students not from #1 ranked college.

#### 3.5 Print the cross tab for prestige_4

In [135]:
print pd.crosstab(df["prestige"], df["admit"]).ix[4]

admit
0    55
1    12
Name: 4, dtype: int64


#### 3.6 Calculate the OR 

In [147]:
OR = (12/55)/[(33+53+28)/(28+95+93)]=0.413

#### 3.7 Write this finding in a sentence

Answer: The odds of getting admitted to grad school for students from #4 ranked college is 0.413 times the odds of admission for students not from #4 ranked college.

## Part 4. Analysis

In [137]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           1           0
1      1  660.0  3.67           0           1           0
2      1  800.0  4.00           0           0           0
3      1  640.0  3.19           0           0           1
4      0  520.0  2.93           0           0           1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [138]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [94]:
train_cols=data[['gre','gpa','prestige_2','prestige_3','prestige_4']]
print train_cols.head()

     gre   gpa  prestige_2  prestige_3  prestige_4
0  380.0  3.61           0           1           0
1  660.0  3.67           0           1           0
2  800.0  4.00           0           0           0
3  640.0  3.19           0           0           1
4  520.0  2.93           0           0           1


#### 4.2 Fit the model

In [145]:
import statsmodels.formula.api as smf
#lm_multi = smf.logit(formula = "admit ~ gre + gpa + C(prestige)", data=df).fit()
lm_multi = smf.logit(formula = "admit ~ gre + gpa + prestige_2 + prestige_3 + prestige_4", data=data).fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [150]:
lm_multi.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sat, 01 Jul 2017   Pseudo R-squ.:                 0.08166
Time:                        10:15:53   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.8769      1.142     -3.393      0.001      -6.116      -1.638
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
==============================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [152]:
#Odds ratios
print(np.exp(lm_multi.params))

Intercept     0.020716
gre           1.002221
gpa           2.180027
prestige_2    0.506548
prestige_3    0.262192
prestige_4    0.211525
dtype: float64


In [153]:
#95% CI Intervals
conf=lm_multi.conf_int()
conf.columns=['2.5%', '97.5%']
print conf

                2.5%     97.5%
Intercept  -6.116077 -1.637631
gre         0.000074  0.004362
gpa         0.127619  1.431056
prestige_2 -1.301337 -0.058936
prestige_3 -2.014579 -0.662776
prestige_4 -2.371624 -0.735197


#### 4.5 Interpret the OR of Prestige_2

Answer: OR of Prestige_2 is approximately 0.5 which means for every one student in the prestige 2 group that gets admitted to graduate school, there are around 2 students in the same group that will not get admitted to graduate school.

#### 4.6 Interpret the OR of GPA

Answer: OR of GPA is approximately 2.18 which means for every 1 point increase in GPA, the odds of getting admitted into grad school is roughly 2.18 times of the original odds.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [76]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [154]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [155]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks=pd.get_dummies(combos['prestige'],  prefix="prestige")
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
print combos.head()
# keep only what we need for making predictions
cols_to_keep = ['gre','gpa','prestige','intercept']
combos=combos[cols_to_keep].join(dummy_ranks.ix[:,'prestige_2':])

     gre       gpa  prestige  intercept
0  220.0  2.260000       1.0        1.0
1  220.0  2.260000       2.0        1.0
2  220.0  2.260000       3.0        1.0
3  220.0  2.260000       4.0        1.0
4  220.0  2.453333       1.0        1.0


#### 5.2 Make predictions on the enumerated dataset

In [159]:
combos['admit_prediction']=lm_multi.predict(combos)
print combos.tail()

       gre       gpa  prestige  intercept  prestige_2  prestige_3  prestige_4  \
395  800.0  3.806667       4.0        1.0           0           0           1   
396  800.0  4.000000       1.0        1.0           0           0           0   
397  800.0  4.000000       2.0        1.0           1           0           0   
398  800.0  4.000000       3.0        1.0           0           1           0   
399  800.0  4.000000       4.0        1.0           0           0           1   

     admit_prediction  
395          0.334286  
396          0.734040  
397          0.582995  
398          0.419833  
399          0.368608  


#### 5.3 Interpret findings for the last 4 observations

Answer: 
Row/observation 396 shows that a candidate with gre score 800, gpa score 4.0 and who is from a prestige 1 school has a 73.40% predicted probability of grad school admission.
Row/observation 397 shows that a candidate with gre score 800, gpa score 4.0 and who is from a prestige 2 school has a 58.30% predicted probability of grad school admission.
Row/observation 398 shows that a candidate with gre score 800, gpa score 4.0 and who is from a prestige 3 school has a 41.98% predicted probability of grad school admission.
Row/observation 399 shows that a candidate with gre score 800, gpa score 4.0 and who is from a prestige 4 school has a 36.86% predicted probability of grad school admission.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.